In [1]:
# Import librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Para obtener lat y lon
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import json
import numpy as np

# Entregable 2 (parte 1)
## Aprendisaje Automatico

**Equipo:** Patrick Murayari, Bruno Weiss, Juan Jara

**Nombre del proyecto**: Aprender 2016-2024 *Entregable 1*

# Descripción del dataset 


* Frecuencia de actualización: Anualmente
* Responsable: Secretaría de Educación 
* Fecha de publicación: 2016
* Fecha de actualización: 2024
* Fuente primaria: Datos abiertos de la Secretaría de Educación
* https://data.educacion.gob.ar/aprender-respuestas-estudiante


* El formato en el que esta presentado son dos archivos .csv
    * Resultados de lengua
    * Resultados de matematica

Esto inicialmente es sospechoso, por eso iniciamos la limpieza de ambos en simultaneo



Este dataset contiene:

* 44+ variables procedentes del aprender de octubre de 2024
* Datos de Ubicacion
* Resultados de evaluaciones en el area de matematica y lengua

**Variables Categoricas:**

- sector, jurisdiccion, departamento, ambito 


**Variables numéricas:**
- 44 preguntas (dirigidas al alumno) de la encuesta aprender 2024 (ya codificadas one-hot).
- Preguntas sobre el nivel de estudio alcanzado por los padres o adulto responsable.
- Nro_Encuestados: la cantidad de encuestados en esa tupla.

* Conjunto de índices:
- NSE (Nivel Socioeconómico) se construyó con las variables: nivel educativo de madre y padre, hacinamiento, percepción de la Asignación Universal por Hijo
por hogar, acceso a internet y acceso a TICs
- Clima escolar se contruyo con las variables "sin marca" y otros motivos desconocidos. 

**Variable objetivo:**
- desempeños en lengua: 'ldesemp_Satisfactorio', 'ldesemp_Por_debajo_del_nivel_básico', 'ldesemp_Básico', 'ldesemp_Avanzado'.
- desempeños en matematica: 'mdesemp_Por_debajo_del_nivel_básico', 'mdesemp_Satisfactorio', 'mdesemp_Básico', 'mdesemp_Avanzado'.



## Hay muchas preguntas en la encuesta pero a continuacion mostramos la mas relevantes y su forma de contestar.

### Preguntas Relevantes del Cuestionario Estudiantes - Nivel Secundario Aprender 2024


*1. Pregunta sobre el Nivel Educativo de la Familia* 

**Pregunta 1: ¿Cuál fue el último nivel educativo alcanzado por tu mamá, papá o persona adulta responsable?**

*a) No fue a la escuela*

*b) Primario*

*c) Secundario*

*d) Terciario, universitario o posgrado*

---

*2. Pregunta sobre Pertenencia Indígena/Originaria de la Familia*

**Pregunta 2: ¿Tu mamá, papá o persona adulta responsable se reconoce de un pueblo indígena u originario o descendiente de una familia indígena u originaria?por ejemplo: Mapuche, Kolla, Toba, Wichi, Diaguita, Guaraní, Qom, otro)**

*a) Si*

*b) No*

---

*3. Pregunta sobre el Reconocimiento Afrodescendiente de la Familia*

**Pregunta 3: ¿Tu mamá, papá o persona adulta responsable se reconoce afrodescendiente o tiene antepasados negros o africanos?**

*a) Si*

*b) No*

---

*4. Pregunta sobre la Repetición de Grado*

**Pregunta 4: ¿Repetiste de grado alguna vez?, ¿en qué ciclo escolar?**

*a) Primaria*

*b) Secundaria: Ciclo Básico*

*c) Secundaria: Ciclo Orientado*

---

*5. Pregunta sobre las Faltas a Clase*

**Pregunta 5: En lo que va del año, ¿cuántas faltas tenés?**

*a) Ninguna falta*

*b) Menos de 5 faltas*

*c) De 5 a 14 faltas*

*d) De 15 a 19 faltas*

*e) De 20 a 29 faltas*

*f) De 30 o mas faltas*

---

*6. Pregunta sobre la Cantidad de Libros en el Hogar*

**Pregunta 6: Aproximadamente, ¿cuántos libros hay en el lugar donde vivís?**


*a) No hay libros en formato papel*

*b) De 1 a 5 libros*

*c) De 6 a 20 libros*

*d) De 21 a 50 libros*

*e) De 51 a 100 libros*

*f) Más de 100 libros*

---

*7. Pregunta sobre el Acceso a Internet y Computadora*

**Pregunta 7: En donde vivís, ¿hay estos objetos o servicios?**


*a) Servicio de streaming por suscripción (Netflix, Amazon, Disney+, otros)*

*b) Canilla con agua potable (apta para el consumo)*

*c) Lavarropas*

*d) Heladera con freezer*

*e) Televisor*

*f) Celular propio*

*g) Tablet*

*h) Lector de libros digitales (e-book)*

*i) Consola de videojuegos (PlayStation, Nintendo Wii, Xbox, PC Gamer, otras)*

*j) Lector de libros digitales (e-book)*

*k) Consola de videojuegos (PlayStation, Nintendo Wii, Xbox, PC Gamer, otras)*

*l) Baño en el interior del lugar donde vivis*

*m) Conexión a Internet*

---

*8. Pregunta sobre la Convivencia Escolar: Relación con la Comunidad Educativa*

**Pregunta 8: Por lo general, ¿cómo es tu relación con las diferentes personas de la escuela?**

*a) Compañeros de curso*

*b) Profesores*	

*c) Equipo directivo*

---

*9. Pregunta sobre Experiencias de Violencia o Bullying en la Escuela*

**Pregunta 9: Durante este año, ¿viviste alguna de las siguientes situaciones con personas de tu escuela?**

*a) Te molestaron o dejaron de lado*

*b) En redes sociales, te amenazaron, agredieron o dijeron mentiras sobre vos*

*c) De forma física o verbal, te amenazaron, agredieron o dijeron mentiras sobre vos*

*d) Te sacaron tus cosas o las rompieron*

*e) Trataste mal a algún compañero*

*f) Viste alguna situación donde agredieran a algún compañero*


---


*10. Pregunta sobre Proyectos Inmediatos al Finalizar el Secundario*

**Pregunta 10: Al finalizar el secundario, ¿qué proyectos inmediatos tenés?**

*a) Trabajar*

*b) Continuar estudios superiores (universitario o no universitario)*

*c) Hacer cursos de capacitación laboral (electricidad, peluquería, manicuría, de huerta, artesanías, del área de construcciones, etc.)*

*d) Ayudar a mi familia con las tareas del hogar*

*e) Producir contenidos para las redes sociales*

*f) Jugar videojuegos para generar dinero*

*g) Realizar trabajos solidarios o comunitarios (participación en comedores, asistencia a personas mayores, acciones religiosas, etc.)*

*h) Desarrollar un emprendimiento personal o familiar*

*i) Desarrollar otras actividades (viajar, hacer deportes, actividades artísticas, etc.)*

*j) Aún no lo decidí*

# Declaracion de ubicaciones de archivos:
- esto es para que no queden desordenados

In [2]:
ruta_archivo_leng = "../data/raw/2022 Base APRENDER - Censal - Secundaria 5-6 año - Agregada - Desempeños de Lengua(in).csv"
ruta_archivo_mate = "../data/raw/2022 Base APRENDER - Censal - Secundaria 5-6 año - Agregada - Desempeños de Matematica(in).csv"

ruta_archivo_leng_pre = "../data/interim/2022_Ap_56Lengcoma.csv"
ruta_archivo_mate_pre = "../data/interim/2022_Ap_56Matecoma.csv"

ruta_archivo_leng_pre2 = "../data/interim/2022_Ap_56Lengceros.csv"
ruta_archivo_mate_pre2 = "../data/interim/2022_Ap_56Mateceros.csv"

ruta_archivo_final = "../data/interim/2022_Ap_56listo.csv"
ruta_archivo_final_latlon = "../data/interim/2022_Ap_56latlon.csv"
ruta_archivo_final_final = "../data/processed/2022_Ap_56final.csv"


# Recodificacion 
* Debido a que el dataset tiene una codificacion latin-1 y
  utiliza el coma como punto decimal, esta funcion reemplaza por punto las comas

In [3]:
def reemplazar_comas_por_puntos(input_csv, output_csv):
    """
    Reemplaza todas las comas por puntos en un archivo CSV con separador ';'
    y guarda el resultado en un nuevo archivo.
    
    :param input_csv: Ruta del archivo CSV original
    :param output_csv: Ruta del archivo CSV de salida
    """
    with open(input_csv, 'r', encoding='latin-1') as f_in:
        contenido = f_in.read()
    
    # Reemplaza todas las comas por puntos
    contenido = contenido.replace(',', '.')
    
    with open(output_csv, 'w', encoding='utf-8') as f_out:
        f_out.write(contenido)

In [4]:
reemplazar_comas_por_puntos(ruta_archivo_leng, ruta_archivo_leng_pre)
reemplazar_comas_por_puntos(ruta_archivo_mate, ruta_archivo_mate_pre)

* Se abre el nuevo archivo generado y se comprueba su estado

In [5]:
df1 = pd.read_csv(ruta_archivo_leng_pre, sep=';', decimal=".", low_memory=False)
df2 = pd.read_csv(ruta_archivo_mate_pre, sep=';', decimal=".", low_memory=False)

In [6]:
df1.head(2)

,jurisdiccion,departamento,sector,ambito,ap01_Blanco,ap01_No_disponible,ap01_Multimarca,ap01_Enero,ap01_Febrero,ap01_Marzo,...,Nivel_Ed_Padre_Secundaria_completa,Nivel_Ed_Padre_Terciario_universitario_posgrado_incompleto,Nivel_Ed_Padre_Terciario_universitario_posgrado_completo,Nivel_Ed_Persona_Resp_No_fue_a_la_escuela,Nivel_Ed_Persona_Resp_Primaria_incompleta,Nivel_Ed_Persona_Resp_Primaria_completa,Nivel_Ed_Persona_Resp_Secundaria_incompleta,Nivel_Ed_Persona_Resp_Secundaria_completa,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
0,Ciudad Autónoma de Buenos Aires,COMUNA 1,Estatal,Urbano,,,,156.4825844,184.2636831,124.1228861,...,345.6860784,76.9290281,503.7215381,5.263809,5.8610716,1.359148,13.4986145,12.9542361,4.1705307,28.5977742
1,Ciudad Autónoma de Buenos Aires,COMUNA 1,Privado,Urbano,,,,115.5770086,99.6601844,121.4561405,...,284.9271666,101.0674483,547.7114879,8.5094742,1.3224264,2.2102623,4.928946,7.4801573,2.5359569,13.1266605


* Como se observa que hay columnas vacias, se prodece a procesarlas para ponerlas 
 con el valor 0 de tipo float, dejando sin cambios las primeras 4 columnas que son categoricas

In [7]:
df1.columns

Index(['jurisdiccion', 'departamento', 'sector', 'ambito', 'ap01_Blanco',
       'ap01_No_disponible', 'ap01_Multimarca', 'ap01_Enero', 'ap01_Febrero',
       'ap01_Marzo',
       ...
       'Nivel_Ed_Padre_Secundaria_completa',
       'Nivel_Ed_Padre_Terciario_universitario_posgrado_incompleto',
       'Nivel_Ed_Padre_Terciario_universitario_posgrado_completo',
       'Nivel_Ed_Persona_Resp_No_fue_a_la_escuela',
       'Nivel_Ed_Persona_Resp_Primaria_incompleta',
       'Nivel_Ed_Persona_Resp_Primaria_completa',
       'Nivel_Ed_Persona_Resp_Secundaria_incompleta',
       'Nivel_Ed_Persona_Resp_Secundaria_completa',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo'],
      dtype='object', length=1229)

In [8]:
def rellenar_y_convertir_columnas_desde_4(df: pd.DataFrame) -> pd.DataFrame:
    """
    Rellena los valores vacíos o con solo espacios con 0 y convierte a float
    todas las columnas desde la 4ta hasta la última del DataFrame.
    Evita el warning de downcasting de pandas.
    """
    columnas = df.columns[4:]
    
    for col in columnas:
        df[col] = df[col].mask(df[col].astype(str).str.strip() == '', 0)
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)
    
    return df

In [9]:
df1 = rellenar_y_convertir_columnas_desde_4(df1)
df2 = rellenar_y_convertir_columnas_desde_4(df2)

In [10]:
df1.head()

,jurisdiccion,departamento,sector,ambito,ap01_Blanco,ap01_No_disponible,ap01_Multimarca,ap01_Enero,ap01_Febrero,ap01_Marzo,...,Nivel_Ed_Padre_Secundaria_completa,Nivel_Ed_Padre_Terciario_universitario_posgrado_incompleto,Nivel_Ed_Padre_Terciario_universitario_posgrado_completo,Nivel_Ed_Persona_Resp_No_fue_a_la_escuela,Nivel_Ed_Persona_Resp_Primaria_incompleta,Nivel_Ed_Persona_Resp_Primaria_completa,Nivel_Ed_Persona_Resp_Secundaria_incompleta,Nivel_Ed_Persona_Resp_Secundaria_completa,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
0,Ciudad Autónoma de Buenos Aires,COMUNA 1,Estatal,Urbano,0.0,0.0,0.0,156.482584,184.263683,124.122886,...,345.686078,76.929028,503.721538,5.263809,5.861072,1.359148,13.498615,12.954236,4.170531,28.597774
1,Ciudad Autónoma de Buenos Aires,COMUNA 1,Privado,Urbano,0.0,0.0,0.0,115.577009,99.660184,121.456141,...,284.927167,101.067448,547.711488,8.509474,1.322426,2.210262,4.928946,7.480157,2.535957,13.126660
2,Ciudad Autónoma de Buenos Aires,COMUNA 10,Estatal,Urbano,0.0,0.0,0.0,76.280651,90.988554,109.889142,...,289.467656,72.806121,226.706185,2.274056,0.981607,3.533785,3.684055,1.417876,1.042957,6.613489
3,Ciudad Autónoma de Buenos Aires,COMUNA 10,Privado,Urbano,0.0,0.0,0.0,108.541738,98.797647,123.833662,...,338.321010,106.824725,478.718275,1.454057,2.561490,1.440209,1.363805,3.846830,1.252726,19.977124
4,Ciudad Autónoma de Buenos Aires,COMUNA 11,Estatal,Urbano,0.0,0.0,0.0,45.374327,55.159915,68.957897,...,224.554457,40.758954,87.550179,0.000000,1.063407,2.699419,4.898296,5.521539,1.928156,3.014935


In [11]:
df2.head()

,jurisdiccion,departamento,sector,ambito,ap01_Blanco,ap01_No_disponible,ap01_Multimarca,ap01_Enero,ap01_Febrero,ap01_Marzo,...,Nivel_Ed_Padre_Secundaria_completa,Nivel_Ed_Padre_Terciario_universitario_posgrado_incompleto,Nivel_Ed_Padre_Terciario_universitario_posgrado_completo,Nivel_Ed_Persona_Resp_No_fue_a_la_escuela,Nivel_Ed_Persona_Resp_Primaria_incompleta,Nivel_Ed_Persona_Resp_Primaria_completa,Nivel_Ed_Persona_Resp_Secundaria_incompleta,Nivel_Ed_Persona_Resp_Secundaria_completa,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple,Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
0,Ciudad Autónoma de Buenos Aires,COMUNA 1,Estatal,Urbano,0.0,0.0,0.0,153.786507,187.956363,129.003955,...,350.017086,79.089308,500.964342,6.628156,4.640560,1.359217,14.830996,15.461703,4.202427,29.629914
1,Ciudad Autónoma de Buenos Aires,COMUNA 1,Privado,Urbano,0.0,0.0,0.0,117.106382,99.742512,123.625104,...,288.514852,100.736452,546.900072,8.867841,1.322426,2.210262,4.741936,7.518041,2.603414,12.631287
2,Ciudad Autónoma de Buenos Aires,COMUNA 10,Estatal,Urbano,0.0,0.0,0.0,78.185469,94.423686,113.264501,...,275.095900,73.573103,240.733811,1.047101,1.160140,3.533965,3.552663,1.276154,1.112545,6.537491
3,Ciudad Autónoma de Buenos Aires,COMUNA 10,Privado,Urbano,0.0,0.0,0.0,112.526113,97.614120,123.268758,...,336.527106,109.613788,481.027817,1.491340,2.611202,1.440209,1.363805,3.693347,1.252726,20.140269
4,Ciudad Autónoma de Buenos Aires,COMUNA 11,Estatal,Urbano,0.0,0.0,0.0,42.352293,57.360521,70.650705,...,221.375849,40.162585,87.561150,0.000000,1.063462,2.159645,6.374843,4.740001,1.928255,3.247019


# Guardado intermedio
* Se crea esta funcion para guardar los .csv

In [12]:
def guardar_df(df: pd.DataFrame, ruta: str):
    """
    Guarda un DataFrame en un archivo CSV en la ruta indicada.

    Parámetros:
    df (pd.DataFrame): DataFrame a guardar.
    ruta (str): Ruta completa del archivo donde se guardará el CSV.
    """
    try:
        df.to_csv(ruta, index=False, encoding='utf-8')
        print(f"Archivo guardado exitosamente en: {ruta}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

In [13]:
guardar_df(df1, ruta_archivo_leng_pre2)
guardar_df(df2, ruta_archivo_mate_pre2)

Archivo guardado exitosamente en: ../data/interim/2022_Ap_56Lengceros.csv
Archivo guardado exitosamente en: ../data/interim/2022_Ap_56Mateceros.csv


In [14]:
#Reapertura de los archivos
df1 = pd.read_csv(ruta_archivo_leng_pre2, low_memory=False)
df2 = pd.read_csv(ruta_archivo_mate_pre2, low_memory=False)

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Columns: 1229 entries, jurisdiccion to Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo
dtypes: float64(1225), object(4)
memory usage: 11.4+ MB


In [16]:
df1.dtypes

jurisdiccion                                                        object
departamento                                                        object
sector                                                              object
ambito                                                              object
ap01_Blanco                                                        float64
                                                                    ...   
Nivel_Ed_Persona_Resp_Primaria_completa                            float64
Nivel_Ed_Persona_Resp_Secundaria_incompleta                        float64
Nivel_Ed_Persona_Resp_Secundaria_completa                          float64
Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple    float64
Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo    float64
Length: 1229, dtype: object

## Se procede a comparar ambos dataset el de lengua y matematica

In [17]:
# Comparar dimensiones (deberían ser iguales)
print("Shape archivo1:", df1.shape)
print("Shape archivo2:", df2.shape)

Shape archivo1: (1213, 1229)
Shape archivo2: (1213, 1229)


In [18]:
print(df1.columns)

Index(['jurisdiccion', 'departamento', 'sector', 'ambito', 'ap01_Blanco',
       'ap01_No_disponible', 'ap01_Multimarca', 'ap01_Enero', 'ap01_Febrero',
       'ap01_Marzo',
       ...
       'Nivel_Ed_Padre_Secundaria_completa',
       'Nivel_Ed_Padre_Terciario_universitario_posgrado_incompleto',
       'Nivel_Ed_Padre_Terciario_universitario_posgrado_completo',
       'Nivel_Ed_Persona_Resp_No_fue_a_la_escuela',
       'Nivel_Ed_Persona_Resp_Primaria_incompleta',
       'Nivel_Ed_Persona_Resp_Primaria_completa',
       'Nivel_Ed_Persona_Resp_Secundaria_incompleta',
       'Nivel_Ed_Persona_Resp_Secundaria_completa',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_incomple',
       'Nivel_Ed_Persona_Resp_Terciario_universitario_posgrado_completo'],
      dtype='object', length=1229)


In [19]:
# Listar las columnas de cada uno
cols1 = set(df1.columns)
cols2 = set(df2.columns)

#print("Columnas en archivo1:", cols1)
#print("Columnas en archivo2:", cols2)

# Columnas que comparten
comunes = cols1.intersection(cols2)
#print("\n✅ Columnas comunes:", comunes)

# Columnas que están en uno pero no en el otro
solo_en_1 = cols1 - cols2
solo_en_2 = cols2 - cols1

print("\n📌 Solo en archivo1:", solo_en_1)
print("📌 Solo en archivo2:", solo_en_2)


📌 Solo en archivo1: {'ldesemp_Satisfactorio', 'ldesemp_Avanzado', 'ldesemp_Básico', 'ldesemp_Por_debajo_del_básico'}
📌 Solo en archivo2: {'mdesemp_Por_debajo_del_básico', 'mdesemp_Básico', 'mdesemp_Satisfactorio', 'mdesemp_Avanzado'}


# Diferencias encontradas

hay columnas que estan en uno de ambos datasets, por lo que le vamos a hacer un join


Pero antes, vamos a ver si difieren en ambos datos


In [20]:
# Columnas comunes (convertir a lista ordenada)
comunes = sorted(set(df1.columns).intersection(set(df2.columns)))


# Filtrar ambos dataframes solo con las columnas comunes
df1 = df1[comunes]
df2 = df2[comunes]

# Ver en qué filas/columnas difieren
diff = (df1 != df2)

# Contar cuántas celdas son distintas
total_diferencias = diff.sum().sum()
#print("Total de diferencias:", total_diferencias)

# Mostrar las filas con diferencias
diferencias = df1[diff.any(axis=1)]
#print("Filas con diferencias:")
#print(diferencias)

# Si querés ver lado a lado:
comparacion = df1.compare(df2)
print("Comparacion")
print(comparacion.head(3))


Comparacion
  NSE_nivel_Alto             NSE_nivel_Bajo             NSE_nivel_Medio  \
            self       other           self       other            self   
0     356.587218  365.074023     216.330643  211.380756      941.114144   
1     475.606752  473.820282      52.421852   52.568357      702.225198   
2     197.006754  219.394688      71.818379   75.469860      636.232620   

              NSE_quintiles_Q1             NSE_quintiles_Q2              ...  \
        other             self       other             self       other  ...   
0  922.154601       296.014329  286.582685       300.279080  290.474503  ...   
1  702.416019        78.327178   78.925493       106.471282  105.514917  ...   
2  606.913834        90.230554   94.450544       151.979682  150.833408  ...   

  migracion_Configuración_familiar_no_migrante              \
                                          self       other   
0                                   988.316922  987.562588   
1                        

# Vista en profundidad

debido a que ambos dataset tinen datos censurados, vamos a ver que tanto difieren

la solucion sera hacer un promedio de las columnas que son iguales


In [21]:
# Supongamos que tus DataFrames se llaman df1 y df2

# Columnas comunes
columnas_comunes = df1.columns.intersection(df2.columns)

# Excluir columnas de tipo object (strings)
columnas_numericas = [col for col in columnas_comunes if pd.api.types.is_numeric_dtype(df1[col])]

# Seleccionamos solo columnas numéricas y hacemos copia
df1_comun = df1[columnas_numericas].copy()
df2_comun = df2[columnas_numericas].copy()

# Convertir números con coma decimal a float
for col in columnas_numericas:
    df1_comun[col] = df1_comun[col].astype(str).str.replace(',', '.').astype(float)
    df2_comun[col] = df2_comun[col].astype(str).str.replace(',', '.').astype(float)

# Restar DataFrames
diferencias = df1_comun - df2_comun

# Estadísticas
pd.set_option('display.max_rows', None)
print(diferencias.describe())
print("Máxima diferencia por columna:\n", diferencias.max())
#print("Máxima diferencia absoluta total:", diferencias.abs().max().max())

       NSE_nivel_Alto  NSE_nivel_Bajo  NSE_nivel_Medio  NSE_quintiles_Q1  \
count     1213.000000     1213.000000      1213.000000       1213.000000   
mean        -0.278921       -0.269213        -0.805010         -0.209558   
std          2.513176        3.545135         7.457881          3.332824   
min        -35.009026      -57.966263      -148.902290        -30.468274   
25%         -0.300704       -0.524096        -1.150872         -0.611284   
50%          0.000000        0.000000         0.000000          0.000000   
75%          0.013494        0.220510         0.594823          0.433813   
max         40.044096       24.596826        49.061931         28.428368   

       NSE_quintiles_Q2  NSE_quintiles_Q3  NSE_quintiles_Q4  NSE_quintiles_Q5  \
count       1213.000000       1213.000000       1213.000000       1213.000000   
mean          -0.171703         -0.361920         -0.226221         -0.383742   
std            3.314935          4.138727          3.983514          3.1

In [22]:

recuento = ["ap01_Abril","ap01_Agosto","ap01_Blanco","ap01_Diciembre",
            "ap01_Enero","ap01_Febrero","ap01_Julio","ap01_Junio",
            "ap01_Marzo","ap01_Mayo","ap01_Multimarca", "ap01_No_disponible",
            "ap01_Noviembre","ap01_Octubre","ap01_Septiembre"]



df1['Nro_Encuestados'] = df1[recuento].sum(axis=1)
df2['Nro_Encuestados'] = df2[recuento].sum(axis=1)

In [23]:
print(df1['Nro_Encuestados'])
print(df2['Nro_Encuestados'])

0        1672.853421
1        1323.052764
2         945.690301
3        1282.697710
4         690.356234
5        1383.277542
6         731.723168
7        1366.922906
8         955.330006
9        2921.358227
10        676.595227
11       1275.608855
12        647.235064
13        653.984179
14        846.480613
15        730.275095
16       1263.606117
17       1192.122066
18       1475.492699
19        873.101616
20        668.274357
21       1483.491180
22       1096.402570
23       1233.658704
24        940.109023
25        995.670207
26       1387.734865
27        994.831820
28       1090.777191
29       1499.188608
30         80.868921
31        279.628276
32        114.152251
33         78.821609
34        403.210129
35        147.184939
36         25.591431
37        140.311695
38         48.111891
39         61.128703
40         21.496802
41         94.536250
42       4665.945814
43       2823.701047
44         22.520460
45        257.735670
46       2898.658863
47       2033

#Esto se hace una sola vez

In [24]:
# Excluir columnas de tipo object (strings)

columnas_numericas = df1.columns.drop(["jurisdiccion","departamento","sector","ambito","Nro_Encuestados"]).tolist()
df1[columnas_numericas] = df1[columnas_numericas].div(df1["Nro_Encuestados"], axis=0)

columnas_numericas = df2.columns.drop(["jurisdiccion","departamento","sector","ambito","Nro_Encuestados"]).tolist()
df2[columnas_numericas] = df2[columnas_numericas].div(df2["Nro_Encuestados"], axis=0)

In [25]:
df1.head()

,NSE_nivel_Alto,NSE_nivel_Bajo,NSE_nivel_Medio,NSE_quintiles_Q1,NSE_quintiles_Q2,NSE_quintiles_Q3,NSE_quintiles_Q4,NSE_quintiles_Q5,Nivel_Ed_Madre_No_fue_a_la_escuela,Nivel_Ed_Madre_Primaria_completa,...,migracion_Configuración_familiar_migrante,migracion_Configuración_familiar_no_migrante,sector,sobreedad_1_año_de_sobreedad_18_años_al_30_de_junio,sobreedad_2_años_de_sobreedad_19_años_al_30_de_junio,sobreedad_3_años_o_más_de_sobreedad_20_años_mas_30Jun,sobreedad_Blanco,sobreedad_Edad_teórica_para_el_año_17_años_al_30_de_junio,sobreedad_Menores_de_17_años,Nro_Encuestados
0,0.213161,0.129318,0.562580,0.176952,0.179501,0.141865,0.140815,0.265927,0.008777,0.071321,...,0.406339,0.590797,Estatal,0.159045,0.075377,0.032148,0.0,0.702001,0.0,1672.853421
1,0.359477,0.039622,0.530761,0.059202,0.080474,0.143451,0.201964,0.444769,0.006095,0.036104,...,0.257166,0.739236,Privado,0.105642,0.027355,0.012979,0.0,0.836379,0.0,1323.052764
2,0.208321,0.075943,0.672771,0.095412,0.160708,0.184975,0.235231,0.280708,0.013721,0.029657,...,0.327693,0.645601,Estatal,0.156427,0.038200,0.011690,0.0,0.771784,0.0,945.690301
3,0.385814,0.013583,0.517669,0.022765,0.043584,0.109955,0.245965,0.494796,0.001912,0.012367,...,0.152071,0.843631,Privado,0.080933,0.015362,0.004429,0.0,0.879590,0.0,1282.697710
4,0.102565,0.076788,0.728369,0.110775,0.179737,0.233524,0.215237,0.168449,0.001939,0.067593,...,0.262784,0.727686,Estatal,0.218551,0.067896,0.017916,0.0,0.675540,0.0,690.356234


In [26]:
df2.head()

,NSE_nivel_Alto,NSE_nivel_Bajo,NSE_nivel_Medio,NSE_quintiles_Q1,NSE_quintiles_Q2,NSE_quintiles_Q3,NSE_quintiles_Q4,NSE_quintiles_Q5,Nivel_Ed_Madre_No_fue_a_la_escuela,Nivel_Ed_Madre_Primaria_completa,...,migracion_Configuración_familiar_migrante,migracion_Configuración_familiar_no_migrante,sector,sobreedad_1_año_de_sobreedad_18_años_al_30_de_junio,sobreedad_2_años_de_sobreedad_19_años_al_30_de_junio,sobreedad_3_años_o_más_de_sobreedad_20_años_mas_30Jun,sobreedad_Blanco,sobreedad_Edad_teórica_para_el_año_17_años_al_30_de_junio,sobreedad_Menores_de_17_años,Nro_Encuestados
0,0.220268,0.127537,0.556385,0.172910,0.175259,0.145151,0.137629,0.273241,0.008862,0.070660,...,0.401016,0.595849,Estatal,0.155960,0.080575,0.031241,0.0,0.702116,0.0,1657.405115
1,0.357990,0.039718,0.530704,0.059631,0.079721,0.141933,0.203988,0.443139,0.005153,0.036370,...,0.255504,0.740666,Privado,0.103786,0.028424,0.012370,0.0,0.837953,0.0,1323.555906
2,0.232288,0.079905,0.642581,0.100001,0.159698,0.181946,0.194718,0.318412,0.012690,0.030160,...,0.335467,0.637866,Estatal,0.171331,0.038005,0.012121,0.0,0.757538,0.0,944.493483
3,0.389604,0.013849,0.512435,0.021875,0.042542,0.108520,0.243479,0.499471,0.001996,0.012468,...,0.154608,0.841192,Privado,0.079997,0.016530,0.004510,0.0,0.879080,0.0,1283.422437
4,0.103477,0.080218,0.730900,0.118111,0.181915,0.229372,0.219420,0.165776,0.001794,0.065112,...,0.271886,0.721274,Estatal,0.214255,0.062481,0.017327,0.0,0.685371,0.0,683.832100


# Al pasarlo a frecuencias vamos a ver si hay diferencias

In [27]:
# Columnas comunes
columnas_comunes = df1.columns.intersection(df2.columns)

# Excluir columnas de tipo object (strings)
columnas_numericas = [col for col in columnas_comunes if pd.api.types.is_numeric_dtype(df1[col])]

# Seleccionamos solo columnas numéricas y hacemos copia
df1_comun = df1[columnas_numericas].copy()
df2_comun = df2[columnas_numericas].copy()

# Convertir números con coma decimal a float
for col in columnas_numericas:
    df1_comun[col] = df1_comun[col].astype(str).str.replace(',', '.').astype(float)
    df2_comun[col] = df2_comun[col].astype(str).str.replace(',', '.').astype(float)

# Restar DataFrames
diferencias = df1_comun - df2_comun

# Estadísticas
pd.set_option('display.max_rows', None)
#print(diferencias.describe())
#print("Máxima diferencia por columna:\n", diferencias.max())
#print("Máxima diferencia absoluta total:", diferencias.abs().max().max())

Máxima diferencia por columna:
 NSE_nivel_Alto                                                      0.034766
NSE_nivel_Bajo                                                      0.155844
NSE_nivel_Medio                                                     0.111864
NSE_quintiles_Q1                                                    0.148611
NSE_quintiles_Q2                                                    0.129098
NSE_quintiles_Q3                                                    0.083333
NSE_quintiles_Q4                                                    0.088889
NSE_quintiles_Q5                                                    0.041176
Nivel_Ed_Madre_No_fue_a_la_escuela                                  0.125000
Nivel_Ed_Madre_Primaria_completa                                    0.059309
Nivel_Ed_Madre_Primaria_incompleta                                  0.119048
Nivel_Ed_Madre_Secundaria_completa                                  0.250000
Nivel_Ed_Madre_Secundaria_incompleta        

* la columna ap22c_Entre_5_y_10_horas_en_la_semana varia un 36% entre ambos data sets




sabiendo que en ambos datasets hay datos disertantes debido a un proceso de 
proteccion de datos personales, vamos a utilizar el promedio para unificar un dataset

In [31]:
#Reapertura de los archivos
df1 = pd.read_csv(ruta_archivo_leng_pre2, low_memory=False)
df2 = pd.read_csv(ruta_archivo_mate_pre2, low_memory=False)
recuento = ["ap01_Abril","ap01_Agosto","ap01_Blanco","ap01_Diciembre",
            "ap01_Enero","ap01_Febrero","ap01_Julio","ap01_Junio",
            "ap01_Marzo","ap01_Mayo","ap01_Multimarca", "ap01_No_disponible",
            "ap01_Noviembre","ap01_Octubre","ap01_Septiembre"]

df1['Nro_Encuestados'] = df1[recuento].sum(axis=1)
df2['Nro_Encuestados'] = df2[recuento].sum(axis=1)



df1['ldesemp_Por_debajo_del_nivel_básico'] = df1['ldesemp_Por_debajo_del_básico']
df1 = df1.drop(columns='ldesemp_Por_debajo_del_básico')
df2['mdesemp_Por_debajo_del_nivel_básico'] = df2['mdesemp_Por_debajo_del_básico']
df2 = df2.drop(columns='mdesemp_Por_debajo_del_básico')

Sena1 = ['ldesemp_Avanzado', 'ldesemp_Por_debajo_del_nivel_básico',
                      'ldesemp_Básico', 'ldesemp_Satisfactorio']
Sena2 = [ 'mdesemp_Básico', 'mdesemp_Por_debajo_del_nivel_básico',
                    'mdesemp_Satisfactorio', 'mdesemp_Avanzado']
df1[Sena1] = df1[Sena1].div(df1["Nro_Encuestados"], axis=0)

df2[Sena2] = df2[Sena2].div(df2["Nro_Encuestados"], axis=0)


In [32]:
df1[Sena1].head(3)

,ldesemp_Avanzado,ldesemp_Por_debajo_del_nivel_básico,ldesemp_Básico,ldesemp_Satisfactorio
0,0.310779,0.194494,0.123318,0.473582
1,0.347806,0.112297,0.134190,0.429554
2,0.280412,0.147907,0.123906,0.478107


In [33]:
# Columnas a excluir de df1
excluir = ["jurisdiccion","departamento","sector","ambito"]

# Columnas comunes (sin las excluidas)
comunes = [c for c in df1.columns.intersection(df2.columns) if c not in excluir]

# Promedio de las comunes
promedio = (df1[comunes] + df2[comunes]) / 2

# Columnas exclusivas de df2
solo_df2 = df2[Sena2]
solo_df1 = df1[Sena1]
exclu = df1[excluir]

# Unificación final: comunes promediadas + exclusivas de df2
resultado = pd.concat([promedio, solo_df2, solo_df1, exclu], axis=1)



In [34]:
resultado.head(3)

,ap01_Blanco,ap01_No_disponible,ap01_Multimarca,ap01_Enero,ap01_Febrero,ap01_Marzo,ap01_Abril,ap01_Mayo,ap01_Junio,ap01_Julio,...,mdesemp_Satisfactorio,mdesemp_Avanzado,ldesemp_Avanzado,ldesemp_Por_debajo_del_nivel_básico,ldesemp_Básico,ldesemp_Satisfactorio,jurisdiccion,departamento,sector,ambito
0,0.0,0.0,0.0,155.134546,186.110023,126.563421,119.860626,118.699505,140.588759,117.011616,...,0.446122,0.0,0.310779,0.194494,0.123318,0.473582,Ciudad Autónoma de Buenos Aires,COMUNA 1,Estatal,Urbano
1,0.0,0.0,0.0,116.341695,99.701348,122.540622,115.699693,112.186181,104.751604,104.782093,...,0.469248,0.0,0.347806,0.112297,0.134190,0.429554,Ciudad Autónoma de Buenos Aires,COMUNA 1,Privado,Urbano
2,0.0,0.0,0.0,77.233060,92.706120,111.576822,70.657364,87.013765,80.989695,57.039804,...,0.409186,0.0,0.280412,0.147907,0.123906,0.478107,Ciudad Autónoma de Buenos Aires,COMUNA 10,Estatal,Urbano


In [35]:

#Recuperando los valores
resultado[Sena1] = resultado[Sena1].mul(resultado["Nro_Encuestados"], axis=0)
resultado[Sena2] = resultado[Sena2].mul(resultado["Nro_Encuestados"], axis=0)



In [36]:
resultado.head()

,ap01_Blanco,ap01_No_disponible,ap01_Multimarca,ap01_Enero,ap01_Febrero,ap01_Marzo,ap01_Abril,ap01_Mayo,ap01_Junio,ap01_Julio,...,mdesemp_Satisfactorio,mdesemp_Avanzado,ldesemp_Avanzado,ldesemp_Por_debajo_del_nivel_básico,ldesemp_Básico,ldesemp_Satisfactorio,jurisdiccion,departamento,sector,ambito
0,0.0,0.0,0.0,155.134546,186.110023,126.563421,119.860626,118.699505,140.588759,117.011616,...,742.851334,0.0,517.486898,323.857102,205.340149,788.575692,Ciudad Autónoma de Buenos Aires,COMUNA 1,Estatal,Urbano
1,0.0,0.0,0.0,116.341695,99.701348,122.540622,115.699693,112.186181,104.751604,104.782093,...,620.958558,0.0,460.253807,148.603691,177.574579,568.431026,Ciudad Autónoma de Buenos Aires,COMUNA 1,Privado,Urbano
2,0.0,0.0,0.0,77.233060,92.706120,111.576822,70.657364,87.013765,80.989695,57.039804,...,386.717992,0.0,265.014924,139.785962,117.102827,451.855411,Ciudad Autónoma de Buenos Aires,COMUNA 10,Estatal,Urbano
3,0.0,0.0,0.0,110.533926,98.205883,123.551210,104.370767,100.433392,112.783954,92.430047,...,535.991337,0.0,382.573674,159.216819,164.785306,645.497431,Ciudad Autónoma de Buenos Aires,COMUNA 10,Privado,Urbano
4,0.0,0.0,0.0,43.863310,56.260218,69.804301,57.455074,75.770319,51.791603,52.100756,...,200.387236,0.0,128.747022,159.700324,110.743366,322.707647,Ciudad Autónoma de Buenos Aires,COMUNA 11,Estatal,Urbano


In [37]:
guardar_df(resultado, ruta_archivo_final)

Archivo guardado exitosamente en: ../data/interim/2022_Ap_56listo.csv


# Transformaciones: 

* Pasamos las columnas jurisdiccion y departamento a coordenadas geograficas
    * Esto es para no tener que hacer codificaciones onehot
    * Tambien sirve para poder introducir directamente la ubicacion del establecimiento educativo.


* Para lograr esto, utilizamos la api de Nominatim
    * Las consultas son cada 5 segundos (86 minutos)
    * Se genera un json para usarlo de cache.
    * Los departamentos enmascarados van a tener el centroide de la provincia
    * Los departamentos que no se encuentren, tambien van a tener el centroide de la provincia. 

In [38]:
#Abrimos el ultimo archivo
df = pd.read_csv(ruta_archivo_final, low_memory=False)

In [39]:
df['jurisdiccion'].head(3)

0    Ciudad Autónoma de Buenos Aires
1    Ciudad Autónoma de Buenos Aires
2    Ciudad Autónoma de Buenos Aires
Name: jurisdiccion, dtype: object

In [40]:

# Inicializar geolocalizador
geolocator = Nominatim(user_agent="mi_app_geocoding")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=5)

# Cargar cache si existe
try:
    with open('cache_geocoding.json', 'r') as f:
        cache = json.load(f)
except FileNotFoundError:
    cache = {}

# Función para obtener coordenadas
def obtener_coordenadas(juris, depar):
    # Si es Enmascadaro, usar solo la provincia
    if depar == "Enmascarado":
        key = f"{juris}"  # solo la provincia
    else:
        key = f"{depar}, {juris}"  # combinación normal

    if key in cache:
        return pd.Series(cache[key])
    
    try:
        location = geocode(key)
        if location:
            coords = [location.latitude, location.longitude]
            cache[key] = coords
            return pd.Series(coords)
        else:
            cache[key] = [None, None]
            return pd.Series([None, None])
    except:
        cache[key] = [None, None]
        return pd.Series([None, None])

# Aplicar función
df[['lat', 'lon']] = df.apply(lambda row: obtener_coordenadas(row['jurisdiccion'], row['departamento']), axis=1)

# Guardar cache en disco
with open('cache_geocoding.json', 'w') as f:
    json.dump(cache, f)


In [41]:
guardar_df(df, ruta_archivo_final_latlon)

Archivo guardado exitosamente en: ../data/interim/2022_Ap_56latlon.csv


- confirmamos que en las ultimas columnas esten los datos de coordenadas:

In [42]:
df.head()

,ap01_Blanco,ap01_No_disponible,ap01_Multimarca,ap01_Enero,ap01_Febrero,ap01_Marzo,ap01_Abril,ap01_Mayo,ap01_Junio,ap01_Julio,...,ldesemp_Avanzado,ldesemp_Por_debajo_del_nivel_básico,ldesemp_Básico,ldesemp_Satisfactorio,jurisdiccion,departamento,sector,ambito,lat,lon
0,0.0,0.0,0.0,155.134546,186.110023,126.563421,119.860626,118.699505,140.588759,117.011616,...,517.486898,323.857102,205.340149,788.575692,Ciudad Autónoma de Buenos Aires,COMUNA 1,Estatal,Urbano,-34.609558,-58.388790
1,0.0,0.0,0.0,116.341695,99.701348,122.540622,115.699693,112.186181,104.751604,104.782093,...,460.253807,148.603691,177.574579,568.431026,Ciudad Autónoma de Buenos Aires,COMUNA 1,Privado,Urbano,-34.609558,-58.388790
2,0.0,0.0,0.0,77.233060,92.706120,111.576822,70.657364,87.013765,80.989695,57.039804,...,265.014924,139.785962,117.102827,451.855411,Ciudad Autónoma de Buenos Aires,COMUNA 10,Estatal,Urbano,-34.627458,-58.503098
3,0.0,0.0,0.0,110.533926,98.205883,123.551210,104.370767,100.433392,112.783954,92.430047,...,382.573674,159.216819,164.785306,645.497431,Ciudad Autónoma de Buenos Aires,COMUNA 10,Privado,Urbano,-34.627458,-58.503098
4,0.0,0.0,0.0,43.863310,56.260218,69.804301,57.455074,75.770319,51.791603,52.100756,...,128.747022,159.700324,110.743366,322.707647,Ciudad Autónoma de Buenos Aires,COMUNA 11,Estatal,Urbano,-34.603006,-58.500717


In [43]:
#Rellenamos las coordenadas vacias

def rellenar_coord_provincia(df):
    """
    Rellena las coordenadas vacías (lat/lon) usando la coordenada de la provincia.
    Guarda y usa cache (cache_geocoding.json).
    """
    geolocator = Nominatim(user_agent="mi_app_geocoding")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

    # Cargar cache si existe
    try:
        with open('cache_geocoding.json', 'r') as f:
            cache = json.load(f)
    except FileNotFoundError:
        cache = {}

    def obtener_coord_provincia(juris):
        if juris in cache:
            return cache[juris]
        try:
            loc = geocode(f"{juris}, Argentina")
            if loc:
                coords = [loc.latitude, loc.longitude]
                cache[juris] = coords
                return coords
            else:
                cache[juris] = [None, None]
                return [None, None]
        except:
            cache[juris] = [None, None]
            return [None, None]

    # Iterar sobre filas donde falten coordenadas
    for idx, row in df[df['lat'].isna() | df['lon'].isna()].iterrows():
        coords = obtener_coord_provincia(row['jurisdiccion'])
        df.loc[idx, ['lat', 'lon']] = coords

    # Guardar cache actualizado
    with open('cache_geocoding.json', 'w') as f:
        json.dump(cache, f)

    return df

In [44]:
df = rellenar_coord_provincia(df)

In [45]:
guardar_df(df, ruta_archivo_final_latlon)

Archivo guardado exitosamente en: ../data/interim/2022_Ap_56latlon.csv


# Codificacion

* la columna sector y ambito van a codificarse de manera binaria

In [46]:
# Codificacion de sector y ambito

# Convertir: 
df['sector_binario'] = df['sector'].map({'Estatal': 1, 'Privado': 0})
df['ambito_binario'] = df['ambito'].map({'Urbano': 1, 'Rural': 0})

# Eliminacion de columnas:

* se eliminan columnas anteriores a la transformacion
- jurisdiccion, departamento, sector y ambito


* vamos a eliminar las columnas de indices, ya que estas estan formuladas
en base a la forma de responder de la encuesta o los niveles de educacion de los tutores.
- se eliminan las columnas asociadas a clima (clima escolar)
- se eliminan las columnas asociadas a NSE (nivel socio economico) 

In [50]:
A_borrar = ['NSE_nivel_Bajo','NSE_nivel_Bajo','NSE_nivel_Alto',
            "NSE_quintiles_Q1","NSE_quintiles_Q2","NSE_quintiles_Q3","NSE_quintiles_Q4","NSE_quintiles_Q5",
           "jurisdiccion","departamento","sector","ambito"]


df.drop(columns=A_borrar, inplace=True)

In [52]:
def revisar_nans(df):
    """
    Verifica si un DataFrame tiene valores NaN y los imprime.
    """
    if df.isna().any().any():  # True si hay al menos un NaN
        print("El DataFrame tiene valores NaN.")
        # Opcional: mostrar cuántos NaN por columna
        #print(df.isna().sum())
    else:
        print("El DataFrame no tiene valores NaN.")
revisar_nans(df)

El DataFrame tiene valores NaN.


In [53]:
def mostrar_filas_nan(df):
    """
    Imprime las filas que contienen al menos un valor NaN en el DataFrame.
    """
    filas_nan = df[df.isna().any(axis=1)]
    if not filas_nan.empty:
        print("Filas con NaN:")
        print(filas_nan)
    else:
        print("No hay filas con NaN.")
mostrar_filas_nan(df)

Filas con NaN:
     ap01_Blanco  ap01_No_disponible  ap01_Multimarca  ap01_Enero  \
562          0.0                 0.0              0.0         0.0   
592          0.0                 0.0              0.0         0.0   

     ap01_Febrero  ap01_Marzo  ap01_Abril  ap01_Mayo  ap01_Junio  ap01_Julio  \
562           0.0         0.0         0.0        0.0         0.0         0.0   
592           0.0         0.0         0.0        0.0         0.0         0.0   

     ...  mdesemp_Satisfactorio  mdesemp_Avanzado  ldesemp_Avanzado  \
562  ...                    NaN               NaN               NaN   
592  ...                    NaN               NaN               NaN   

     ldesemp_Por_debajo_del_nivel_básico  ldesemp_Básico  \
562                                  NaN             NaN   
592                                  NaN             NaN   

     ldesemp_Satisfactorio        lat        lon  sector_binario  \
562                    NaN -27.471148 -58.989909               1   
592  

In [54]:
# eliminar filas que tengan al menos un NaN
df = df.dropna()

In [55]:
revisar_nans(df)

El DataFrame no tiene valores NaN.


# Listo

Ya tenemos un dataset limpio para el año 2024.
La idea de este notebook es utilizarlo para todos los años restantes y hacer una union entre todos

In [56]:
guardar_df(df, ruta_archivo_final_final)

Archivo guardado exitosamente en: ../data/processed/2022_Ap_56final.csv
